#### (1) Runnable 이란?

* LangChain에서 **다양한 컴포넌트(예: LLM, 프롬프트, 파서 등)를 연결하고 실행하기 위한 기본 인터페이스**

* 여러 컴포넌트를 연결 -> 복잡한 데이터 처리 **파이프라인 구축** 가능
    * 예시: prompt -> LLM에 전달 -> LLM의 출력을 파서로 전달
    * `invoke`, `batch`, `stream` 등의 메서드를 통해 실행
    * 각 Runnabel은 입력 데이터를 받아 처리하고, 그 결과를 다음 Runnable에 전달하는 역할

* **LCEL과의 관계** : LCEL은 Runnable을 기반으로 Chain을 보다 쉽게 구성할 수 있도록 하는 선언적인 언어
    * 
    * `RunnablePassthrough`: 입력을 변경하지 않거나 추가 키를 더하여 전달
        * `RunnablePassthrough()`: 단독으로 호출 -> 단순히 입력을 받아 그대로 전달
        * `RunnablePassthrough()`: 입력을 받아 assign 함수에 전달된 추가 인수를 추가
    * `RunnableParallel`: 여러 Runnable을 병렬로 실행
    * `RunnableSequence`: `Runnable`의 시퀀스를 정의함
    * `RunnableLambda`: 사용자 정의 함수를 `Runnable`로 래핑함
    * 
    * 위와 같이 다양한 Runnable 클래스 조합으로 체인 구축 가능

---

* 기본 환경 설정

In [ ]:
# 기본 모듈 임포트
import os
import asyncio
from dotenv import load_dotenv

# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보 로드
load_dotenv()                   # true

In [ ]:
# 환경 변수 확인하기

# 마스킹 처리 함수 정의
def mask_key(key: str, visible_count: int = 2) -> str:
    if not key or len(key) <= visible_count:
        return '*' * len(key)
    return key[:visible_count] + '*' * (len(key) - visible_count)

# 환경변수 불러오기
api_key = os.getenv("GOOGLE_API_KEY")
if not api_key:
    raise ValueError("GOOGLE_API_KEY 환경 변수가 설정되지 않았습니다.")

# 마스킹된 형태로 출력
print(f"GOOGLE_API_KEY: {mask_key(api_key)}")           # GOOGLE_API_KEY: AI*************************************

In [ ]:
# LangSmith 추적 설정 (https://smith.langchain.com)

"""
- !pip install -qU langsmith
- !pip install -qU langchain-teddynote
    -> 제미나이와 poetry와의 의존성 충돌로 langchain_teddy 설치 X 
    -> langsmith로 진행
"""
# LangSmith 추적을 위한 라이브러리 임포트
from langsmith import traceable         # @traceable 데코레이터 사용 시

# LangSmith 환경 변수 확인

print("\n--- LangSmith 환경 변수 확인 ---")
langchain_tracing_v2 = os.getenv('LANGCHAIN_TRACING_V2')
langchain_project = os.getenv('LANGCHAIN_PROJECT')
langchain_api_key_status = "설정됨" if os.getenv('LANGCHAIN_API_KEY') else "설정되지 않음" # API 키 값은 직접 출력하지 않음

if langchain_tracing_v2 == "true" and os.getenv('LANGCHAIN_API_KEY') and langchain_project:
    print(f"✅ LangSmith 추적 활성화됨 (LANGCHAIN_TRACING_V2='{langchain_tracing_v2}')")
    print(f"✅ LangSmith 프로젝트: '{langchain_project}'")
    print(f"✅ LangSmith API Key: {langchain_api_key_status}")
    print("  -> 이제 LangSmith 대시보드에서 이 프로젝트를 확인해 보세요.")
else:
    print("❌ LangSmith 추적이 완전히 활성화되지 않았습니다. 다음을 확인하세요:")
    if langchain_tracing_v2 != "true":
        print(f"  - LANGCHAIN_TRACING_V2가 'true'로 설정되어 있지 않습니다 (현재: '{langchain_tracing_v2}').")
    if not os.getenv('LANGCHAIN_API_KEY'):
        print("  - LANGCHAIN_API_KEY가 설정되어 있지 않습니다.")
    if not langchain_project:
        print("  - LANGCHAIN_PROJECT가 설정되어 있지 않습니다.")

<small>

* 셀 출력
    * --- LangSmith 환경 변수 확인 ---
    * ✅ LangSmith 추적 활성화됨 (LANGCHAIN_TRACING_V2='true')
    * ✅ LangSmith 프로젝트: 'L***************'
    * ✅ LangSmith API Key: 설정됨
    *   -> 이제 LangSmith 대시보드에서 이 프로젝트를 확인해 보세요.

In [ ]:
# LangChain 및 Google GenAI 모델 관련 모듈 임포트
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough, RunnableLambda
from langchain_core.output_parsers import StrOutputParser
from langchain_google_genai import ChatGoogleGenerativeAI           # Google GenAI 임포트


print("\n--- LangChain 체인 설정 ---")

template = "{topic}에 대해 3문장으로 설명해줘."                           # 템플릿 정의

prompt = PromptTemplate.from_template(template)                     # 프롬프트를 프롬프트템플릿 객체로 생성

try:
    model = ChatGoogleGenerativeAI(                                 # 모델 호출
        model="gemini-1.5-flash",
        temperature=0.1,
    )
    print("✅ Google GenAI 모델 초기화 성공.")
except Exception as e:                                              # 디버깅 메시지
    print(f"❌ Google GenAI 모델 초기화 실패: {e}")
    print("  -> GOOGLE_API_KEY 환경 변수가 올바르게 설정되었는지 확인하세요.")
    
# 출력 파서
output_parser = StrOutputParser()

# 체인 구성
chain = prompt | model | output_parser                              # 프롬프트, 모델, 출력 파서 연결 -> 체인 구성
print("✅ LangChain LCEL 체인 구성 완료.")

<small>

* 셀 출력
    * --- LangChain 체인 설정 ---
    * ✅ Google GenAI 모델 초기화 성공.
    * ✅ LangChain LCEL 체인 구성 완료.

---

#### (2) **`RunnablePassthrough`**


* **`RunnablePassthrough`**
  
  * **입력 데이터를 변형하지 않고 그대로 통과**시키거나, **필요에 따라 일부 키/값을 추가**해 입력에 함께 전달할 수 있는 Runnable 객체

  * Q. 언제 쓸까?
    * A1. 여러 체인에서, **입력 데이터의 특정 부분을 그대로 넘기고 싶을 때**
    * A2. **기존 입력데이터 + 추가 데이터 조합이 필요**할 때
      * 예시: `assign` 활용
    
  * 특징
      * `.invoke()` -> **입력을 그대로 반환**
      * `.assign()` -> 입력 dict에 **원하는 계산 결과를 병합**해줌

In [ ]:
# 교재와 같이 시도

# prompt, model 생성
prompt = PromptTemplate.from_template("{num}의 10는?")

try:
    model = ChatGoogleGenerativeAI(                                 # 기본 모델 호출
        model="gemini-1.5-flash",
        temperature=0.1,
    )
    print("✅ Google GenAI 모델 초기화 성공.")
    print("---")
    
except Exception as e:                                              # 디버깅 메시지
    print(f"❌ Google GenAI 모델 초기화 실패: {e}")
    print("  -> GOOGLE_API_KEY 환경 변수가 올바르게 설정되었는지 확인하세요.")
    print("---")

# Chain 생성
chain = prompt | model

# Chain 실행
chain.invoke({"num":5})

<small>

* 
    * 셀 출력
        * ✅ Google GenAI 모델 초기화 성공.
        * ---
        * `AIMessage(content='5의 10승은 5를 10번 곱한 값입니다.  즉, 5 x 5 x 5 x 5 x 5 x 5 x 5 x 5 x 5 x 5 = **9,765,625** 입니다.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-1.5-flash', 'safety_ratings': []}, id='run--a7f7c69f-7477-42fd-a200-93d5f949b716-0', usage_metadata={'input_tokens': 7, 'output_tokens': 67, 'total_tokens': 74, 'input_token_details': {'cache_read': 0}})`


In [ ]:
# 1개의 변수만 템플릿에 포함하고 있다면 값만 전달하는 것도 가능
chain.invoke(5)

<small>

* 
    * 셀 출력
        * `AIMessage(content='5의 10승은 5 x 5 x 5 x 5 x 5 x 5 x 5 x 5 x 5 x 5 = **9,765,625** 입니다.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-1.5-flash', 'safety_ratings': []}, id='run--2b1a9028-cdd9-4748-be61-0f20d3dab11f-0', usage_metadata={'input_tokens': 7, 'output_tokens': 51, 'total_tokens': 58, 'input_token_details': {'cache_read': 0}})`

* **`RunnablePassthrough()`**: 입력 값을 **그대로 받아 전달**
    * `invoke()` 메소드와 사용
    * chain 구성 연습

In [ ]:
# RunnablePassthrough 관련 모듈 임포트
from langchain_core.runnables import RunnablePassthrough

# RunnablePassthrough()_1
RunnablePassthrough().invoke({"num: 10"})                           # {'num: 10'}

# RunnablePassthrough()_2
result = RunnablePassthrough().invoke({"num": 10})
print(result)                                                       # {'num': 10}

In [ ]:
# RunnablePassthrough -> Chain 구성하기

runnablepassthrough_chain = {"num" : RunnablePassthrough()} | prompt | model

# dict 값이 RunnablePassthrough()로 변경됨
runnablepassthrough_chain.invoke(10)

<small>

* 
    * 셀 출력
        * `AIMessage(content='10의 10승은 10,000,000,000 (100억) 입니다.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-1.5-flash', 'safety_ratings': []}, id='run--2c9d7372-8b1c-4da2-928d-da5bfe77de31-0', usage_metadata={'input_tokens': 8, 'output_tokens': 32, 'total_tokens': 40, 'input_token_details': {'cache_read': 0}})`

* **`RunnablePassthrough()`**: 입력 값에 **추가 키 더하여 전달 가능**
    * `assign()` 함수에 전달된 추가 인수 전달
    * chain 구성 연습

In [ ]:
# RunnablePassthrough.assign()_1
RunnablePassthrough.assign().invoke({"num": 1})         # {'num': 1}

# RunnablePassthrough.assign()_2
result_assign = RunnablePassthrough.assign().invoke({"num": 1})
print(result_assign)                                      # {`num``: 1}

In [ ]:
# RunnablePassthrough.assign()_3 - 새롭게 할당된 key/value 쌍 병합
# 입력 키: num, 할당(assign) 키: new_num

(RunnablePassthrough.assign(new_num=lambda x: x["num"] * 3)).invoke({"num": 1})             # {'num': 1, 'new_num': 3}

In [ ]:
# RunnablePassthrough.assign()_4 - assign에 함수 추가해보기

result_assign2 = RunnablePassthrough().assign(new_num=lambda x: x["num"] * 3)
print(result_assign2.invoke({"num": 5}))                                                    # {'num': 5, 'new_num': 15}

---

#### (3) **`RunnableParallel`**


* **`RunnableParallel`**
  
  * 입력 데이터를 **여러 개의 Runnable 체인에 병렬로 전달**하고, 각 실행 **결과를 dict형태로 반환**

  * Q. 언제 쓸까?
    * A. 여러 모델 or 다른 연산을 동시에 돌려 **다중 결과가 필요**한 경우
      * 예시_1: 한 입력으로 여러 LLM 질의
      * 예시_2: 전처리 + 후처리 결과 동시 반환 등
    
  * 특징
      * **dict형태** -> **Key별로 runnable 매핑 가능**
      * **결과 또한 Key별로 추출**
      * 병렬 실행은 내부적으로 `await`/`gather`로 처리, **I/O 차원에선 동시에 실행**

In [ ]:
# RunnableParallel 모듈 임포트
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

# RunnableParallel 인스턴스 생성 -> 여러 인스턴스 병렬 실행 가능
runnable_parallel = RunnableParallel(
    # RunnablePassthrough() = 'passed' 키워드 인자로 전달 -> 입력된 데이터를 그대로 통과
    passed=RunnablePassthrough(),
    
    # 'extra' 키워드 인자로 RunnablePassthrough.assign('mult' 람다 함수) -> 딕셔너리의 'num' 키에 해당하는 값을 3배로 증가
    extra=RunnablePassthrough.assign(mult=lambda x: x["num"] * 3),
    
    # 람다 함수 = 'modified' 키워드 인자 -> 입력된 딕셔너리의 'num' 키에 해당하는 값에 1 더하기
    modified=lambda x: x["num"] + 1,
)

# {'num': 1} 딕셔너리 입력 -> invoke 메소드로 호출해보기
runnable_parallel.invoke({"num": 1})                                

# {'passed': {'num': 1}, 'extra': {'num': 1, 'mult': 3}, 'modified': 2} 

In [ ]:
import time                                                                                 #시간 측정을 위해 임포트

# 모델 초기화 (gemini-2.5-flash-lite 사용)
try:
    model2 = ChatGoogleGenerativeAI(                                 
        model="gemini-2.5-flash-lite",
        temperature=0.1,
    )
    print("✅ Google GenAI 모델 초기화 성공.")
    print("---")
except Exception as e:                                              
    print(f"❌ Google GenAI 모델 초기화 실패: {e}")
    print("  -> GOOGLE_API_KEY 환경 변수가 올바르게 설정되었는지 확인하세요.")
    print("---")
    exit() # 모델 초기화 실패 시 프로그램 종료

# 프롬프트 구성
prompt_capital = PromptTemplate.from_template("{country}의 수도는?")
prompt_area = PromptTemplate.from_template("{country}의 면적은?")

###################################################
# 동기적 실행과 RunnableParaller 실행을 모두 진행하여 비교 #
###################################################

# 동기적 실행을 위한 개별 체인 (RunnableParallel 없이 순차적으로 호출)
chain_capital_sync = {"country": RunnablePassthrough()} | prompt_capital | model2
chain_area_sync = {"country": RunnablePassthrough()} | prompt_area | model2

# RunnableParallel을 사용한 병렬 체인
# model2를 사용하도록 변경
combined_chain_parallel = RunnableParallel(capital=chain_capital_sync, area=chain_area_sync)

# --- 동기적(순차적) 실행 시간 측정 ---
print("--- 동기적(순차적) 실행 시작 ---")
start_time_sync = time.time()

# 두 체인을 순차적으로 호출
capital_result_sync = chain_capital_sync.invoke("대한민국")
area_result_sync = chain_area_sync.invoke("대한민국")

end_time_sync = time.time()

print(f"수도 결과: {capital_result_sync}")                                     
print(f"면적 결과: {area_result_sync}")                                         
print(f"동기적 실행 시간: {end_time_sync - start_time_sync:.4f} 초")
print("---")

# --- RunnableParallel (병렬) 실행 시간 측정 ---
print("\n--- RunnableParallel (병렬) 실행 시작 ---")
start_time_parallel = time.time()

# RunnableParallel 체인 호출
result_parallel = combined_chain_parallel.invoke("대한민국")

end_time_parallel = time.time()

# 결과 출력 (딕셔너리 형태이므로 직접 접근)
print(f"병렬 실행 결과 - 수도: {result_parallel['capital']}")                    
print(f"병렬 실행 결과 - 면적: {result_parallel['area']}")                      
print(f"병렬 실행 시간: {end_time_parallel - start_time_parallel:.4f} 초")
print("---")

<small>

* 
    * 셀 출력
        * ✅ Google GenAI 모델 초기화 성공.
        * ---
        * --- 동기적(순차적) 실행 시작 ---
        * 수도 결과: content='대한민국의 수도는 **서울**입니다.' additional_kwargs={} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash-lite', 'safety_ratings': []} id='run--aee20300-a869-415d-a425-69e21ccd30ed-0' usage_metadata={'input_tokens': 7, 'output_tokens': 10, 'total_tokens': 17, 'input_token_details': {'cache_read': 0}}
        * 면적 결과: content='대한민국의 면적은 약 **100,410 제곱킬로미터**입니다.\n\n이는 세계에서 109번째로 넓은 면적이며, 한반도 전체 면적의 약 45%에 해당합니다.' additional_kwargs={} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash-lite', 'safety_ratings': []} id='run--e9164940-80c1-4e10-bd51-cd6203de12b1-0' usage_metadata={'input_tokens': 8, 'output_tokens': 55, 'total_tokens': 63, 'input_token_details': {'cache_read': 0}}
        * 동기적 실행 시간: 1.9750 초
        * ---

        * --- RunnableParallel (병렬) 실행 시작 ---
        * 병렬 실행 결과 - 수도: content='대한민국의 수도는 **서울**입니다.' additional_kwargs={} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash-lite', 'safety_ratings': []} id='run--485dc267-1502-4323-a326-501811f157b5-0' usage_metadata={'input_tokens': 7, 'output_tokens': 10, 'total_tokens': 17, 'input_token_details': {'cache_read': 0}}
        * 병렬 실행 결과 - 면적: content='대한민국의 면적은 약 **100,410 제곱킬로미터**입니다.\n\n이는 세계적으로 볼 때 중간 정도의 크기이며, 한반도 전체 면적의 약 45%에 해당합니다.' additional_kwargs={} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash-lite', 'safety_ratings': []} id='run--ba33162d-bd75-4ca4-9c9c-e38534d4afc5-0' usage_metadata={'input_tokens': 8, 'output_tokens': 52, 'total_tokens': 60, 'input_token_details': {'cache_read': 0}}
        * 병렬 실행 시간: 1.2327 초
        * ---

---

#### (4) **`RunnableLambda`**

* **`RunnableLambda`**
  
  * 파이썬의 **any 함수(람다 등)를 체인 내에서 실행**해 **결과를 반환**하는 Runnable 객체

  * Q. 언제 쓸까?
    * A1. 체인 중간에 직접적 데이터 가공/전처리가 필요할 때
    * A2. 입력값 동적 계산/포맷팅/파싱 로직이 필요할 때
    
  * 특징
      * 나만의 커스텀 연산을 체인에 쉽게 삽입 가능
      * `invoke()` 시 입력값 받아 함수 적용, 반환값 반환

In [ ]:
# RunnalbeLambda_1

# 필요한 임포트
from datetime import datetime

def get_today(a):
    # 오늘 날짜 가져오기
    return datetime.today().strftime("%b-%d")

# 출력하기
get_today(None)                                      # 'Jul-30'

In [ ]:
# RunnalbeLambda_2

# 필요한 모듈 임포트
from langchain_core.runnables import RunnableLambda

# 입력값의 제곱을 반환하는 lambda 적용
runnable_lambda = RunnableLambda(lambda x: x["num"] ** 2)
print(runnable_lambda.invoke({"num": 7}))                  # 49

In [ ]:
# RunnableLambda_3 - Chain 생성

def get_today(a):
    return datetime.today().strftime("%b-%d")


# 필요한 모듈 임포트
from langchain_core.runnables import RunnableLambda, RunnablePassthrough

# 프롬프트 생성
prompt_lambda = PromptTemplate.from_template(
    "{today}가 생일인 유명인 (n) 명을 나열하세요. 생년월일을 표기하세요."
)

# model = gemini-2.5-flash-lite 사용
try:
    model2 = ChatGoogleGenerativeAI(                                 # 모델 호출
        model="gemini-2.5-flash-lite",
        temperature=0.1,
    )
    print("✅ Google GenAI 모델 초기화 성공.")
    print("---")
except Exception as e:                                              # 디버깅 메시지
    print(f"❌ Google GenAI 모델 초기화 실패: {e}")
    print("  -> GOOGLE_API_KEY 환경 변수가 올바르게 설정되었는지 확인하세요.")
    print("---")


# chain 생성
chain_lambda =(
    {"today":RunnableLambda(get_today), "n":RunnablePassthrough()}
    | prompt_lambda
    | model2
    | StrOutputParser()
)

# 체인 객체의 구성 출력 (현재 교재에서 보여주는 단계)
print("--- Chain 구성 정보 ---")
print(chain_lambda)
print("---")

# 체인 실행 및 실제 결과 출력
print("\n--- 체인 실행 결과 ---")
try:
    # "n"에 해당하는 값을 invoke() 메서드에 딕셔너리 형태로 전달합니다.
    # 예: 3명의 유명인
    result = chain_lambda.invoke({"n": 3})
    print(result)
except Exception as e:
    print(f"❌ 체인 실행 중 오류 발생: {e}")

<small>

* 
    * 셀 출력 
        * ✅ Google GenAI 모델 초기화 성공.

        ---

    * --- Chain 구성 정보 ---
        * chain_lambda 객체 자체의 **표현** = 어떻게 구성되었는지를 표혀줌
        * `RunnabelLambda`, `PromptTemplate`, `ChatGoogleGenerativeAI`, `StrOutParser` 같은 컴포넌트들을 성공적으로 연결해서 하나의 체인을 만들었음을 보여줌 
          * `first={today: RunnableLambda(get_today), n: RunnablePassthrough()}` = 체인의 첫 번째 단계에서 입력(n)과 get_today 함수(today)를 어떻게 처리하는지 보여줌
          * `middle=[PromptTemplate(input_variables=['today'], input_types={}, partial_variables={}, template='{today}가 생일인 유명인 (n) 명을 나열하세요. 생년월일을 표기하세요.'), ChatGoogleGenerativeAI(model='models/gemini-2.5-flash-lite', google_api_key=SecretStr('**********'), temperature=0.1, client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x11f2079b0>, default_metadata=(), model_kwargs={})]` = 프롬프트 템플릿과 실제 언어 모델(ChatGoogleGenerativeAI)이 어떻게 연결되어 있는지 나타냄
          * `last=StrOutputParser()` = 마지막으로 StrOutputParser()를 사용하여 모델의 응답을 문자열로 파싱하는 단계를 보여줌
        
        ---

    * --- 체인 실행 결과 ---
        * 7월 30일에 생일을 맞은 유명인 5명을 생년월일과 함께 알려드리겠습니다.

        * 1.  **헨리 포드 (Henry Ford)**
        *     *   생년월일: 1863년 7월 30일

        * 2.  **에밀리 브론테 (Emily Brontë)**
        *     *   생년월일: 1818년 7월 30일

        * 3.  **폴 앤더슨 (Paul Anderson)**
        *     *   생년월일: 1932년 7월 30일

        * 4.  **리사 쿤트로 (Lisa Kudrow)**
        *     *   생년월일: 1963년 7월 30일

        * 5.  **크리스토퍼 멜로니 (Christopher Meloni)**
        *     *   생년월일: 1961년 7월 30일

-----

<small>

* 
    * **문제점** 발견 : 3명 출력 요구 -> 5명이 출력
      * 주요 원인1. model의 **자유성**
          * 현재 설정: temperature = 0.1 -> randomness or creativity 낮음 
      * 주요 원인2. **프롬프트의 모호성**
          * **정확히**라는 강조가 부족했을 수 있음
          * `유명인 (n) 명을 나열하세요` -> `(n)` = 모델에게 `n`이 **단순한 플레이스홀더**인지, 아니면 **정확히 지켜야 할 숫자 제한**인지 **혼동**을 줄 수 있음
            * 즉, 모델이 **(n)의 의미를 숫자 제한으로 정확히 이해하지 못했을 가능성**
          * 프롬프트가 길어지거나 복잡해질수록, 모델은 특정 숫자 제한보다는 **전체적인 내용과 의미를 파악**하는 데 집중할 필요가 있음
      * 주요 원인3. 데이터의 다양성
          * 모델이 학습한 데이터에는 **몇 명**이라는 숫자가 명확히 지켜지지 않은 목록이나 설명이 많을 수 있음
          * 이러한 학습 데이터의 편향이 모델의 응답에 영향을 미칠 수도 있음
      * 주요 원인4. model의 최적화 목표
          * temperature가 낮을 때 모델은 자신이 학습한 데이터에서 "가장 자연스럽고 그럴듯한" 답변을 내놓으려고 함 
              * 만약 학습 데이터에 **유명인 나열**이라는 프롬프트에 대해 3명보다 **5명을 나열하는 경우가 더 흔하거나** 모델이 **5명을 나열하는 패턴을 더 강력하게 학습**했다면, temperature가 낮을수록 오히려 그 강력한 패턴을 따르려 할 가능성이 있음
              * 3명으로 제한하는 것보다 5명으로 제한하지 않고 나열하는 것이 모델 입장에서는 더 "자연스러운" 답변이라고 판단했을 것
          * 내부적 선호도
              * 모델 내부적으로 특정 유형의 정보(**이름 목록**)를 생성할 때 **일정 개수 이상을 선호하는 경향**이 있을 수 있음
              * 낮은 온도는 이런 내부적인 선호도를 더 강하게 표출하게 만들 수도 있음

    * **모델의 temperature가 낮아도 프롬프트가 모호하면 다른 내부적 우선순위를 따르게 됨**
    ---

    * 해결 방안
        * temperature 더 낮추기? -> **X**
            * temperature가 낮다고 해서 지시를 무조건 따르는 것이 아님
            * model의 내부적인 학습 패턴과 프롬프트의 명확성과 함께 고려해야 함
        * **프롬프트 개선 필요**

----

In [ ]:
# # RunnableLambda_4 - Chain 생성 + 프롬프트 개선

from datetime import datetime
import re
from operator import itemgetter

from langchain_core.runnables import RunnableLambda, RunnablePassthrough, RunnableParallel
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_google_genai import ChatGoogleGenerativeAI


# 날짜 함수
def get_today(_): return datetime.today().strftime("%b-%d")

# 프롬프트 템플릿
prompt_lambda2 = PromptTemplate.from_template("""
당신은 데이터를 엄격히 다루는 어시스턴트입니다.

규칙:
- '{today}가 생일인 유명인 {n_val}명:'으로 시작
- 한 문장에 한 명: 1. 이름 (생년월일)
- 예시만 따르기, 설명・공백・인삿말 없이

예시:
{today}가 생일인 유명인 3명:
1. 알베르트 아인슈타인 (1879년 3월 14일)
2. 김연아 (1990년 9월 5일)
3. BTS의 정국 (1997년 9월 1일)

---

{today}가 생일인 유명인 {n_val}명을 위와 같은 형식으로 출력하세요.
""".strip())

# 모델 초기화
model2 = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash-lite",
    temperature=0.1,
)

# 후처리 함수
def post_process(data):
    text = data["text"]
    n = data["n_val"]
    if isinstance(n, dict):
        n = n.get("n_val", 1)
    try:
        limit = int(n)
    except:
        print(f"[ERROR] 잘못된 n_val 값: {n} ({type(n)})")
        return ""

    lines = [
        l.strip()
        for l in text.strip().split("\n")
        if re.match(r"^\d+\.\s.+\(\d{4}", l.strip())
    ]
    return "\n".join(lines[:limit])

# 체인 구성
chain_lambda2 = (
    {
        "today": RunnableLambda(get_today),
        "n_val": RunnablePassthrough()
    }
    | RunnableParallel(
        text=(prompt_lambda2 | model2 | StrOutputParser()),
        n_val=itemgetter("n_val")  # 🔥 핵심 수정: 중첩 dict 방지
    )
    | RunnableLambda(post_process)
)

# 실행
num = 3
print("✅ 실행 중...")
result = chain_lambda2.invoke({"n_val": num})
print("\n🎯 최종 결과:\n" + result)


# 체인 객체의 구성 출력 (현재 교재에서 보여주는 단계)
print("\n--- Chain 구성 정보 ---")
print(chain_lambda2)
print("---")

# 체인 실행 및 실제 결과 출력
print("\n--- 체인 실행 결과 ---")
try:
    # "n"에 해당하는 값을 invoke() 메서드에 딕셔너리 형태로 전달합니다.
    # 예: 3명의 유명인
    result = chain_lambda2.invoke({"n_val": 3})
    print(result)
except Exception as e:
    print(f"❌ 체인 실행 중 오류 발생: {e}")


<small>

* 
    * 셀 출력
        * ✅ 실행 중...

        * 🎯 최종 결과:
        * 1. 헨리 포드 (1863년 7월 30일)
        * 2. 닐 암스트롱 (1930년 8월 5일)
        * 3. 톰 행크스 (1956년 7월 9일)

        ---

        * --- Chain 구성 정보 ---
        * first=`{today: RunnableLambda(get_today), n_val: RunnablePassthrough()}`
            * **chin_lambda2의 구성 자체**를 보여줌
        * middle=`[{text: PromptTemplate(input_variables=['n_val', 'today'], input_types={}, partial_variables={}, template="당신은 데이터를 엄격히 다루는 어시스턴트입니다.\n\n규칙:\n- '{today}가 생일인 유명인 {n_val}명:'으로 시작\n- 한 문장에 한 명: 1. 이름 (생년월일)\n- 예시만 따르기, 설명・공백・인삿말 없이\n\n예시:\n{today}가 생일인 유명인 3명:\n1. 알베르트 아인슈타인 (1879년 3월 14일)\n2. 김연아 (1990년 9월 5일)\n3. BTS의 정국 (1997년 9월 1일)\n\n---\n\n{today}가 생일인 유명인 {n_val}명을 위와 같은 형식으로 출력하세요.") | ChatGoogleGenerativeAI(model='models/gemini-2.5-flash-lite', google_api_key=SecretStr('**********'), temperature=0.1, client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x1081a8eb0>, default_metadata=(), model_kwargs={})| StrOutputParser(),n_val: RunnableLambda(itemgetter('n_val'))}]`
            * 프롬프트 템플릿과 실제 언어 모델(ChatGoogleGenerativeAI)이 어떻게 연결되어 있는지 나타냄
        * last=`RunnableLambda(post_process)`
            * 마지막으로 StrOutputParser()를 사용하여 모델의 응답을 문자열로 파싱하는 단계를 보여줌

        ---

        * --- 체인 실행 결과 ---
        * 1. 헨리 포드 (1863년 7월 30일)
        * 2. 닐 암스트롱 (1930년 8월 5일)
        * 3. 톰 행크스 (1956년 7월 9일)

#### (5) **`itemgetter`**

* **`operator.itemgetter`** 
  * 개념
    * 파이썬의 표준 라이브러리인 operator 모듈에 있는 함수
    * **딕셔너리**나 **리스트** 같은 객체에서 **특정 키(key) 또는 인덱스(index)에 해당하는 값**을 효율적으로 추출하기 위한 **호출 가능한(callable)** 객체를 반환

  * Q. 언제 쓸까?
    * A1. **딕셔너리**나 **리스트**처럼 여러 값이 모여 있는 데이터에서 **특정 키나 인덱스에 해당하는 값** 하나 또는 여러 개를 **효율적으로 꺼내고 싶을 때 사용**
        * 딕셔너리 예시: {'name': '앨리스', 'age': 30}
        * 리스트 예시: ['사과', '바나나', '오렌지']
    * A2. 특히 **데이터를 가공**하거나, **특정 필드만** 다음 단계로 넘겨야 할 때

  * 특징
    * **간결하고 명확**
        * `lambda x: x['key']` 같은 복잡하게 람다 함수 사용할 필요 X
        * `itemgetter('key')` -> **직관적으로 특정 값 호출 가능**
    * **재사용성 및 효율성**
        *  **한 번 만들어진 itemgetter 객체는 여러 데이터에 반복적으로 적용 가능**
        *  **C 언어로 구현** -> 람다 함수보다 약간 더 빠를 수 있음

In [ ]:
# itemgetter_1

####################################
# 1. 필요한 라이브러리 및 클래스, 모듈 임포트
####################################
# 필요한 라이브러리 임포트 (기타 반복되는 임포트 생략)
from operator import itemgetter
from langchain_core.prompts import ChatPromptTemplate               # 채팅 프롬프트 템플릿을 위한 클래스 임포트

####################################
# 2. 입력 가공 및 준비
####################################

# 문장의 길이를 반환하는 함수
def length_function(text):
    return len(text)


# 두 문장의 길이를 곱한 값을 반환하는 내부(helper)함수
def _multiple_length_function(text1, text2):
    return len(text1) * len(text2)


# _multiple_length_function 함수를 사용하여 두 문장의 길이를 곱한 값을 반환하는 함수
# 딕셔너리 형태의 입력 값을 처리하도록 설계됨
def multiple_length_function(_dict):
    return _multiple_length_function(_dict["text1"], _dict["text2"])

####################################
# 3. LLM 호출
####################################

# 프롬프트 템플릿 정의: {a}와 {b} 값을 받아 질문 생성
prompt_itemgetter = ChatPromptTemplate.from_template("{a} + {b} 는 무엇인가요?")

# 모델 초기화 (gemini-1.5-flash)
try:
    model = ChatGoogleGenerativeAI(
        model="gemini-1.5-flash", # 빠른 응답을 위한 Gemini 1.5 Flash 모델 사용
        temperature=0.1, # 일관된 답변을 위해 낮은 온도 설정
    )
    print("✅ Google GenAI 모델 초기화 성공.")
except Exception as e:
    print(f"❌ Google GenAI 모델 초기화 실패: {e}")
    print("  -> GOOGLE_API_KEY 환경 변수가 올바르게 설정되었는지 확인하세요.")
    print("---")
    exit() # 모델 초기화 실패 시 프로그램 종료

# 기본적인 chain 구성 예시
chain1 = prompt_itemgetter | model          # 이 예시에서 사용되지는 않지만 체인 구성의 기본적인 틀을 보여줌

# main chain 생성 : 입력 가공 -> 프롬프트 채우기 -> 모델 호출
"""
이 체인은 입력 값을 가공해 프롬프트 템플릿에 전달하여 완성된 질문을 언어 모델로 호출하여 다시 전달한 후 답변을 얻어내는 과정을 보여줍니다.
    1. 입력 가공 : 'word1'과 'word2' 두 개의 문자열 입력을 받아 처리합니다.
        - 'a' 값
            - 'word1'의 길이
            - 입력 딕셔너리 -> 'word1' 추출 -> 앞서 정의한 def length_function으로 길이 계산
        -'b' 값
            - 'word1' 길이와 'word2' 길이를 곱한 값
            - 파이썬 내장 라이브러리 함수인 itemgetter 사용
            - 입력 딕셔너리에서 'word1'과 'word2'를 추출하여 {'text1': word1, 'text2': word2} 형태의 딕셔너리 생성
            - 앞서 정의한 def multiple_length_function으로 두 길의 곱 계산

    2. 가공된 {"a": 값, "b": 값} 딕셔너리를 prompt 템플릿에 전달
        - {"a" : word1의 길이, "b": word1 길이 * word2 길이}
        - dict 형태 -> 질문이 완성되면 model에게 전달
        
    3. model 호출 -> 최종 답변 받기

즉, 이 체인은 두 개의 단어를 입력받아 그 단어들의 길이를 계산하고, 이를 변형하여 새로운 숫자로 만든 뒤, 그 숫자들을 조합한 질문을 LLM에게 던져 답을 얻는 로직을 보여줍니다.
chain1은 단순히 프롬프트와 모델을 연결한 기본적인 예시이며, chain_itemgetter가 핵심 로직을 담고 있습니다.
"""
chain_itemgetter = (
    {
        "a": itemgetter("word1") | RunnableLambda(length_function),         # 'a': 입력 딕셔너리 'word1' 추출 -> length_function으로 길이 계산
        "b": {"text1": itemgetter("word1"), "text2": itemgetter("word2")}   # 'b': 입력 딕셔너리 'word1'과 'word2'를 추출 -> {'text1': word1, 'text2': word2} 딕셔너리 생성   
        | RunnableLambda(multiple_length_function),                         # 위 딕셔너리를 multiple_length_function에 전달하여 두 길이의 곱 계산
    }
    | prompt_itemgetter                     # 가공된 {"a": 값, "b": 값} 딕셔너리를 prompt 템플릿에 전달
    | model                                 # 완성된 질문을 언어 모델에 전달하여 답변 얻기
)


<small>

* 
    * 셀 출력
        * ✅ Google GenAI 모델 초기화 성공.

In [68]:
# chain_itemgetter 실행

if __name__ == "__main__":
    print("\n--- 체인 실행 시작 ---")
    try:
        result = chain_itemgetter.invoke({"word1": "hello", "word2": "world"})
        print(f"과정: {result}")
        print(f"최종 답변: {result.content}")
    except NameError:
        print("❌ 'chain_itemgetter'가 정의되지 않았습니다. 이전 셀을 먼저 실행해 주세요.")
    except Exception as e:
        print(f"❌ 체인 실행 중 오류 발생: {e}")
    print("--- 체인 실행 종료 ---")


--- 체인 실행 시작 ---
과정: content='5 + 25 = 30 입니다.' additional_kwargs={} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-1.5-flash', 'safety_ratings': []} id='run--d1e1f6aa-cac4-4484-9766-3b1adc6dae55-0' usage_metadata={'input_tokens': 12, 'output_tokens': 12, 'total_tokens': 24, 'input_token_details': {'cache_read': 0}}
최종 답변: 5 + 25 = 30 입니다.
--- 체인 실행 종료 ---


<small>

* 
    * 셀 출력
        * --- 체인 실행 시작 ---
        * 과정: `content='5 + 25 = 30 입니다.' additional_kwargs={} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-1.5-flash', 'safety_ratings': []} id='run--d1e1f6aa-cac4-4484-9766-3b1adc6dae55-0' usage_metadata={'input_tokens': 12, 'output_tokens': 12, 'total_tokens': 24, 'input_token_details': {'cache_read': 0}}`
        * 최종 답변: 5 + 25 = 30 입니다.
        * --- 체인 실행 종료 ---

In [69]:
# 교재와 동일하게 출력해보기

chain_itemgetter.invoke({"word1": "hello", "word2": "world"})

AIMessage(content='5 + 25 = 30 입니다.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-1.5-flash', 'safety_ratings': []}, id='run--7b4398e3-3cc6-4d83-9a1d-a95ecf1f8707-0', usage_metadata={'input_tokens': 12, 'output_tokens': 12, 'total_tokens': 24, 'input_token_details': {'cache_read': 0}})

<small>

* 
    * AIMessage 객체 출력의 차이점

    | 특성 / 출력 방식    | `chain.invoke(...)` 직접 출력 (`__repr__` 호출) | `print(result)` (`__str__` 호출)                  |
    | :------------------ | :--------------------------------------------- | :------------------------------------------------ |
    | **목적** | 개발자/디버깅용                                  | 사용자 친화적/코드 내 출력용                      |
    | **표현 내용** | 객체의 모든 속성 (Content, 메타데이터, ID 등 상세 정보) | 객체의 핵심 내용 (주로 `content`만)              |
    | **형식** | `AIMessage(content='...', response_metadata={...}, id='...', ...)` | `5 + 25 = 30 입니다.` (간결한 텍스트)             |
    | **가독성** | 상세하지만 다소 복잡                               | 간결하고 읽기 쉬움                                |
    | **주요 사용처** | 객체의 상태/정보 전체를 확인해야 할 때             | 최종 결과를 보여주거나 로그로 남길 때             |